In [1]:
import pandas as pd
import time

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

from tqdm.autonotebook import tqdm

import nlpaug.augmenter.word as naw

import tensorflow as tf

from keras.models import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

import re
import string
import sys
from collections import Counter
import os, re, csv, math, codecs

from numpy import array
from numpy import asarray
from numpy import zeros
import numpy as np
from numpy import where

import gensim

from matplotlib import pyplot
import matplotlib.pyplot as plt

import seaborn as sns

from roundtrip import backtranslate

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize

C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\2246951324.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt to C:\Users\sadeep-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\sadeep-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\sadeep-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def read_data(file_path,notnull_field=""):
    data=pd.read_csv(file_path)
    if(notnull_field!=""):
        data = data[pd.notnull(data[notnull_field])]
    return data 

In [3]:
def augment_data_bert(data_set,save_to):  
    result=data_set['category_id'].value_counts()
    num_classes=result.count()
    print("num classes:" ,num_classes)
    majority_class_count=result.max()
    print("num majority class count:" ,majority_class_count)
    for i in range(num_classes):
        sub_set=data_set[data_set['category_id'] == i].reset_index(drop=True)
        main_set=data_set[data_set['category_id'] != i].reset_index(drop=True)
        sample_size=majority_class_count-sub_set.count()['category_id']
        print("sample size:" ,sample_size)
        st = time.time()
        if(sample_size>0):
            col_name =sub_set.columns
            aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
            print(col_name)
            for j in tqdm(range(0,sample_size), desc ="Augmenting Data"):
                #sub_set=shuffle(sub_set)
                #print("review selected",sub_set[col_name[5]][3] )
                review=aug.augment(sub_set[col_name[5]][j])
                category=sub_set[col_name[9]][j]
                categoryId=sub_set[col_name[14]][j]
                term=sub_set[col_name[10]][j]
                sentiment=sub_set[col_name[13]][j]
                new_data = pd.DataFrame([[review,category,term,sentiment,categoryId]],columns=['review','category','term','sentiment','category_id'])
                sub_set=sub_set.append(new_data,ignore_index=True)
            data_set=main_set.append(sub_set,ignore_index=True)
            data_set['category_id'].value_counts()
        et = time.time()
        elapsed_time = (et - st)/60
        print('Execution time:', elapsed_time, 'minutes')
    data_set.to_csv(save_to, index=False)

In [4]:
def augment_g_data_bert(data_set,save_to):     
    result=data_set['category_id'].value_counts()
    num_classes=result.count()
    print("num classes:" ,num_classes)
    majority_class_count=result.max()
    print("num majority class count:" ,majority_class_count)
    for i in range(num_classes):
        sub_set=data_set[data_set['category_id'] == i].reset_index(drop=True)
        main_set=data_set[data_set['category_id'] != i].reset_index(drop=True)
        sample_size=majority_class_count-sub_set.count()['category_id']
        print("sample size:" ,sample_size)
        if(sample_size>0):
            col_name =sub_set.columns
            aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
            print(col_name)
            for j in tqdm(range(0,sample_size), desc ="Augmenting Data"):
                #sub_set=shuffle(sub_set)
                #print("review selected",sub_set[col_name[5]][3] )
                review=aug.augment(sub_set[col_name[5]][j])
                category=sub_set[col_name[9]][j]
                categoryId=sub_set[col_name[11]][j]
                sentiment=sub_set[col_name[10]][j]
                new_data = pd.DataFrame([[review,category,sentiment,categoryId]],columns=['review','category','sentiment','category_id'])
                sub_set=sub_set.append(new_data,ignore_index=True)
            data_set=main_set.append(sub_set,ignore_index=True)
            data_set['category_id'].value_counts()
    data_set.to_csv(save_to, index=False)

In [5]:
def augment_data_rtt(data_set,save_to,lang):
    result=data_set['category_id'].value_counts()
    num_classes=result.count()
    print("num classes:" ,num_classes)
    majority_class_count=result.max()
    print("num majority class count:" ,majority_class_count)
    for i in range(num_classes):
        sub_set=data_set[data_set['category_id'] == i].reset_index(drop=True)
        main_set=data_set[data_set['category_id'] != i].reset_index(drop=True)
        sample_size=majority_class_count-sub_set.count()['category_id']
        print("sample size:" ,sample_size)
        st = time.time()
        if(sample_size>0):
            col_name =sub_set.columns
            for j in tqdm(range(0,sample_size), desc ="Augmenting Data"):
                #sub_set=shuffle(sub_set)
                #print("review selected",sub_set[col_name[5]][3] )
                review=backtranslate(sub_set[col_name[5]][j],lang)
                category=sub_set[col_name[9]][j]
                categoryId=sub_set[col_name[14]][j]
                term=sub_set[col_name[10]][j]
                sentiment=sub_set[col_name[13]][j]
                new_data = pd.DataFrame([[review,category,term,sentiment,categoryId]],columns=['review','category','term','sentiment','category_id'])
                sub_set=sub_set.append(new_data,ignore_index=True)
            data_set=main_set.append(sub_set,ignore_index=True)
            data_set['category_id'].value_counts()
        et = time.time()
        elapsed_time = (et - st)/60
        print('Execution time:', elapsed_time, 'minutes')
    data_set.to_csv(save_to, index=False)

In [6]:
def augment_g_data_rtt(data_set,save_to,lang):
    result=data_set['category_id'].value_counts()
    num_classes=result.count()
    print("num classes:" ,num_classes)
    majority_class_count=result.max()
    print("num majority class count:" ,majority_class_count)
    for i in range(num_classes):
        sub_set=data_set[data_set['category_id'] == i].reset_index(drop=True)
        main_set=data_set[data_set['category_id'] != i].reset_index(drop=True)
        sample_size=majority_class_count-sub_set.count()['category_id']
        print("sample size:" ,sample_size)
        if(sample_size>0):
            col_name =sub_set.columns
            for j in tqdm(range(0,sample_size), desc ="Augmenting Data"):
                #sub_set=shuffle(sub_set)
                #print("review selected",sub_set[col_name[5]][3] )
                review=backtranslate(sub_set[col_name[5]][j],lang)
                category=sub_set[col_name[9]][j]
                categoryId=sub_set[col_name[11]][j]
                sentiment=sub_set[col_name[10]][j]
                new_data = pd.DataFrame([[review,category,sentiment,categoryId]],columns=['review','category','sentiment','category_id'])
                sub_set=sub_set.append(new_data,ignore_index=True)
            data_set=main_set.append(sub_set,ignore_index=True)
            data_set['category_id'].value_counts()
    data_set.to_csv(save_to, index=False)

In [7]:
def pro_aug():
    domain="productivity"
    base_path=f"C:\\Users\\sadeep-pc\\Desktop\\tenfolddataset\\{domain}\\"
    
    for i in range(10):
        print(f"iteration: {i+1}")
        st = time.time()
        source_flie=base_path+f"train{i+1}.csv"
        save_to=base_path+f"train{i+1}_bert.csv"
        training_data=read_data(source_flie,"category")
        augment_data_bert(training_data,save_to)
        et = time.time()
        elapsed_time = (et - st)/3600
        print('Execution time:', elapsed_time, 'hours')
    

In [8]:
def pro_rtt_aug():
    domain="productivity"
    base_path=f"C:\\Users\\sadeep-pc\\Desktop\\tenfolddataset\\{domain}\\"
     
    languages = ["de", "cn", "ja","tr"]
    
    for x in languages:
        print(f"language: {x}")
        st = time.time()
        for i in range(10):
            print(f"iteration: {i+1}")
            source_flie=base_path+f"train{i+1}.csv"
            save_to=base_path+f"train{i+1}_{x}.csv"
            training_data=read_data(source_flie,"category")
            augment_data_rtt(training_data,save_to,x)
        et = time.time()
        elapsed_time = (et - st)/3600
        print('Execution time:', elapsed_time, 'hours')
    

In [9]:
def social_aug():
    domain="social"
    base_path=f"C:\\Users\\sadeep-pc\\Desktop\\tenfolddataset\\{domain}\\"
    
    for i in range(10):
        print(f"iteration: {i+1}")
        st = time.time()
        source_flie=base_path+f"train{i+1}.csv"
        save_to=base_path+f"train{i+1}_bert.csv"
        training_data=read_data(source_flie,"category")
        augment_data_bert(training_data,save_to)
        et = time.time()
        elapsed_time = (et - st)/3600
        print('Execution time:', elapsed_time, 'hours')

In [10]:
def social_rtt_aug():
    domain="social"
    base_path=f"C:\\Users\\sadeep-pc\\Desktop\\tenfolddataset\\{domain}\\"

    languages = ["de", "cn", "ja","tr"]
    
    for x in languages:
        print(f"language: {x}")
        st = time.time()
        for i in range(10):
            print(f"iteration: {i+1}")
            source_flie=base_path+f"train{i+1}.csv"
            save_to=base_path+f"train{i+1}_{x}.csv"
            training_data=read_data(source_flie,"category")
            augment_data_rtt(training_data,save_to,x)
        et = time.time()
        elapsed_time = (et - st)/3600
        print('Execution time:', elapsed_time, 'hours')

In [11]:
def game_aug():
    domain="game"
    base_path=f"C:\\Users\\sadeep-pc\\Desktop\\tenfolddataset\\{domain}\\"
    
    for i in range(10):
        print(f"iteration: {i+1}")
        st = time.time()
        source_flie=base_path+f"train{i+1}.csv"
        save_to=base_path+f"train{i+1}_bert.csv"
        training_data=read_data(source_flie,"category")
        augment_g_data_bert(training_data,save_to)
        et = time.time()
        elapsed_time = (et - st)/3600
        print('Execution time:', elapsed_time, 'hours')

In [12]:
def game_rtt_aug():
    domain="game"
    base_path=f"C:\\Users\\sadeep-pc\\Desktop\\tenfolddataset\\{domain}\\"
     
    languages = ["de", "cn", "ja","tr"]
    
    for x in languages:
        print(f"language: {x}")
        st = time.time()
        for i in range(10):
            print(f"iteration: {i+1}")
            source_flie=base_path+f"train{i+1}.csv"
            save_to=base_path+f"train{i+1}_{x}.csv"
            training_data=read_data(source_flie,"category")
            augment_g_data_rtt(training_data,save_to,x)
        et = time.time()
        elapsed_time = (et - st)/3600
        print('Execution time:', elapsed_time, 'hours')

In [13]:
game_aug()

iteration: 1
num classes: 10
num majority class count: 1301
sample size: 1233
Index(['domain', 'app', 'review_id', 'sentence_id', 'title', 'review',
       'sentence', 'rating', 'is_opinion', 'category', 'sentiment',
       'category_id'],
      dtype='object')


Augmenting Data:   0%|          | 0/1233 [00:00<?, ?it/s]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   0%|          | 1/1233 [00:06<2:03:12,  6.00s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   0%|          | 2/1233 [00:10<1:43:11,  5.03s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   0%|          | 3

Augmenting Data:   2%|▏         | 25/1233 [01:46<1:21:28,  4.05s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   2%|▏         | 26/1233 [01:52<1:35:34,  4.75s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   2%|▏         | 27/1233 [01:58<1:41:10,  5.03s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   2%|

Augmenting Data:   4%|▍         | 50/1233 [03:41<1:35:31,  4.84s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   4%|▍         | 51/1233 [03:43<1:16:13,  3.87s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   4%|▍         | 52/1233 [03:47<1:17:05,  3.92s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   4%|

Augmenting Data:   6%|▌         | 75/1233 [05:30<1:26:54,  4.50s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   6%|▌         | 76/1233 [05:35<1:25:59,  4.46s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   6%|▌         | 77/1233 [05:39<1:25:35,  4.44s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:   6%|

...

Augmenting Data:  42%|████▏     | 416/993 [26:16<36:36,  3.81s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  42%|████▏     | 417/993 [26:20<35:38,  3.71s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  42%|████▏     | 418/993 [26:24<39:09,  4.09s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  42%|████▏ 

Augmenting Data:  45%|████▍     | 442/993 [27:57<38:27,  4.19s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  45%|████▍     | 443/993 [28:02<39:31,  4.31s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  45%|████▍     | 444/993 [28:06<39:46,  4.35s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  45%|████▍ 

Augmenting Data:  47%|████▋     | 468/993 [29:37<27:38,  3.16s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  47%|████▋     | 469/993 [29:39<24:56,  2.86s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  47%|████▋     | 470/993 [29:41<21:36,  2.48s/it]C:\Users\sadeep-pc\AppData\Local\Temp\ipykernel_12932\1764640094.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_set=sub_set.append(new_data,ignore_index=True)
Augmenting Data:  47%|████▋ 

Augmenting Data:  50%|████▉     | 494/993 [31:29<31:48,  3.83s/it]


KeyboardInterrupt: 